### Yield From - Throwing Exceptions

We can throw exceptions into a generator using the throw() method

- Works with delegation as well

The delegator does not intercept the exception

The subgenerator can handle the exception (or not)

As the exception is being propagated up (if chosen to), the delegator may now handle the exception (silence or propagate up)

#### Code Examples

In [1]:
class CloseCoroutine(Exception):
    pass

In [2]:
def echo():
    try:
        while True:
            received = yield
            print(received)
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/ or GeneratorExit thrown')

In [3]:
e = echo()
next(e)

In [4]:
e.throw(CloseCoroutine)

StopIteration: coro was closed

In [6]:
e = echo()
next(e)

In [7]:
e.close()

closed method was called/ or GeneratorExit thrown


In [8]:
e = echo()
next(e)
e.throw(GeneratorExit)

closed method was called/ or GeneratorExit thrown


StopIteration: 

In [9]:
def delegator():
    result = yield from echo()
    yield 'subgen closed and returned:', result
    print('delegator closing...')

In [10]:
d = delegator()
next(d)

In [11]:
d.send('hello')

hello


In [12]:
d.throw(CloseCoroutine)

('subgen closed and returned:', 'coro was closed')

In [13]:
class IgnoreMe(Exception):
    pass

In [14]:
def echo():
    try:
        while True:
            try:
                received = yield
                print(received)
            except IgnoreMe:
                yield "I'm ignoring you!"
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/ or GeneratorExit thrown')
        return 'from a GeneratorExit'

In [15]:
d = delegator()
next(d)

In [16]:
d.send('hello')

hello


In [17]:
result = d.throw(IgnoreMe, 1000)

In [18]:
result

"I'm ignoring you!"

In [19]:
d.send('rocks!')

In [20]:
d.send('yeet')

yeet


In [21]:
d.close()

closed method was called/ or GeneratorExit thrown


In [22]:
def echo():
    output = None
    try:
        while True:
            try:
                received = yield output
                print(received)
            except IgnoreMe:
                output = "I'm ignoring you!"
            else:
                output = None
    except CloseCoroutine:
        return 'coro was closed'
    except GeneratorExit:
        print('closed method was called/ or GeneratorExit thrown')
        return 'from a GeneratorExit'

In [23]:
d = delegator()
next(d)

In [24]:
d.send('python')

python


In [25]:
d.throw(IgnoreMe)

"I'm ignoring you!"

In [26]:
d.send('rocks!')

rocks!


In [27]:
d.close()

closed method was called/ or GeneratorExit thrown


In [28]:
def echo():
    while True:
        received = yield
        print(received)

In [29]:
def delegator():
    yield from echo()

In [30]:
d = delegator()
next(d)

In [31]:
d.throw(ValueError)

ValueError: 

In [32]:
def delegator():
    try:
        yield from echo()
    except ValueError:
        print('delegator got the value error')

In [33]:
d = delegator()
next(d)

In [34]:
d.throw(ValueError)

delegator got the value error


StopIteration: 

In [35]:
def delegator():
    while True:
        try:
            yield from echo()
        except ValueError:
            print('delegator got the value error')

In [36]:
d = delegator()

In [37]:
next(d)

In [38]:
d.send('Python')

Python


In [39]:
d.throw(ValueError)

delegator got the value error


In [40]:
d.send('rocks')

rocks


In [41]:
d.throw(IgnoreMe)

IgnoreMe: 

In [42]:
class WriteAverage(Exception):
    pass

In [44]:
def averager(out_file):
    total = 0
    count = 0
    average = None
    with open(out_file, 'w') as f:
        f.write('count,average\n')
        while True:
            try:
                received = yield average
                total += received
                count += 1
                average = total / count
            except WriteAverage:
                if average is not None:
                    print('saving average to file:', average)
                    f.write(f'{count},{average}\n')

In [45]:
avg = averager('sample.csv')
next(avg)

In [46]:
avg.send(1)
avg.send(2)

1.5

In [47]:
avg.throw(WriteAverage)

saving average to file: 1.5


1.5

In [48]:
avg.send(3)

2.0

In [49]:
avg.send(2)

2.0

In [50]:
avg.throw(WriteAverage)

saving average to file: 2.0


2.0

In [51]:
avg.close()

In [52]:
with open('sample.csv') as f:
    for row in f:
        print(row.strip())

count,average
2,1.5
4,2.0


In [53]:
def delegator():
    yield from averager('sample.csv')

In [61]:
d = delegator()
next(d)

In [62]:
d.send(1)

1.0

In [63]:
d.send(2)

1.5

In [64]:
d.send(3)

2.0

In [65]:
d.send(4)

2.5

In [66]:
d.throw(WriteAverage)

saving average to file: 2.5


2.5

In [67]:
d.send(5)

3.0

In [68]:
d.throw(WriteAverage)

saving average to file: 3.0


3.0

In [70]:
d.close()

In [71]:
with open('sample.csv') as f:
    for row in f:
        print(row.strip())

count,average
4,2.5
5,3.0
